In [0]:
import numpy as np
import tensorflow as tf
from tensorflow.keras import layers
tf.enable_eager_execution()

In [0]:
class RNN(tf.keras.Model):
  def __init__(self):
    super(RNN, self).__init__()
    self.embed = layers.Embedding(10000, 32)
    self.rnn = layers.SimpleRNN(64)
    self.dense = layers.Dense(2, activation = 'softmax')
  def __call__(self, x, training=False, mask=None):
    x = self.embed(x)
    x = self.rnn(x)
    x = self.dense(x)
    return x

In [0]:
@tf.function
def train_step(model, inputs, labels, loss_object, optimizer, train_loss, train_accuracy):
  with tf.GradientTape() as tape:
    predictions = model(inputs)
    loss = loss_object(labels, predictions)
  gradients = tape.gradient(loss, model.trainable_variables)
  optimizer.apply_gradients(zip(gradients, model.trainable_variables))
  train_loss(loss)
  train_accuracy(labels, predictions)

@tf.function
def test_step(model, inputs, labels, loss_object, test_loss, test_accuracy):
  predictions = model(inputs)
  t_loss = loss_object(labels, predictions)
  test_loss(t_loss)
  test_accuracy(labels, predictions)

In [0]:
(train_x, train_y), (test_x, test_y) = tf.keras.datasets.imdb.load_data(num_words = 10000)

x_train = tf.keras.preprocessing.sequence.pad_sequences(train_x, padding='pre', value=0, maxlen=32)
x_test = tf.keras.preprocessing.sequence.pad_sequences(test_x, padding='pre', value=0, maxlen=32)

train_ds = tf.data.Dataset.from_tensor_slices((x_train, train_y)).shuffle(10000).batch(32)
test_ds = tf.data.Dataset.from_tensor_slices((x_test, test_y)).batch(32)

In [0]:
model = RNN()

loss_object = tf.keras.losses.SparseCategoricalCrossentropy()
optimizer = tf.keras.optimizers.Adam()

train_loss = tf.keras.metrics.Mean(name='train_loss')
train_accuracy = tf.keras.metrics.SparseCategoricalAccuracy(name='train_accuracy')
test_loss = tf.keras.metrics.Mean(name='test_loss')
test_accuracy = tf.keras.metrics.SparseCategoricalAccuracy(name='test_accuracy')

In [12]:
for epoch in range(20):
  for inputs, labels in train_ds:
    train_step(model, inputs, labels, loss_object, optimizer, train_loss, train_accuracy)
  for t_inputs, t_labels in test_ds:
    test_step(model, t_inputs, t_labels, loss_object, test_loss, test_accuracy)
  template = 'Epoch {}, Loss: {}, Accuracy: {}, Test Loss: {}, Test Accuracy: {}'
  print(template.format(epoch+1, train_loss.result(), train_accuracy.result()*100, test_loss.result(), test_accuracy.result()*100))
  train_loss.reset_states()
  train_accuracy.reset_states()
  test_loss.reset_states()
  test_accuracy.reset_states()

select: 1
Epoch 1, Loss: 0.5257608890533447, Accuracy: 72.58399963378906, Test Loss: 0.48404985666275024, Test Accuracy: 77.5
Epoch 2, Loss: 0.3563168942928314, Accuracy: 84.86800384521484, Test Loss: 0.5219104290008545, Test Accuracy: 76.74800109863281
Epoch 3, Loss: 0.22418439388275146, Accuracy: 91.21199798583984, Test Loss: 0.6083431243896484, Test Accuracy: 75.81199645996094
Epoch 4, Loss: 0.14787061512470245, Accuracy: 94.2439956665039, Test Loss: 0.7501523494720459, Test Accuracy: 72.1520004272461
Epoch 5, Loss: 0.09982160478830338, Accuracy: 96.4000015258789, Test Loss: 1.1330878734588623, Test Accuracy: 67.01200103759766
Epoch 6, Loss: 0.07758289575576782, Accuracy: 97.0999984741211, Test Loss: 1.0343408584594727, Test Accuracy: 71.59199523925781
Epoch 7, Loss: 0.06187044829130173, Accuracy: 97.76399993896484, Test Loss: 1.1115020513534546, Test Accuracy: 71.59199523925781
Epoch 8, Loss: 0.046047624200582504, Accuracy: 98.37200164794922, Test Loss: 1.149409532546997, Test Accu

ValueError: ignored